# Car ride-share potential in mid-size U.S. cities from geographic spread

## (notebook 3: CITY_INDEX=14)

Third notebook for the IBM Data Science Specialization on Coursera, which contains the consolidated methods and functions from the first two notebooks, to serve as a template to be used per-city.



## Installs (if needed) and imports

See notesbooks 1 and 2 for details.

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math
import json

try:
    from geopy.geocoders import Nominatim
    print(Nominatim)
except:
    !conda install -c conda-forge geopy=1.18.1 --yes
    
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    import folium
    print(folium)
except:
    !conda install -c conda-forge folium=0.8.0 --yes
    
import folium
import time
    
try:
    import pandas_profiling
    print(pandas_profiling)
except:
    !conda install -c conda-forge pandas-profiling=1.4.1 --yes
    
import pandas_profiling

<class 'geopy.geocoders.osm.Nominatim'>
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.8.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  40.54 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  27.82 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  30.81 MB/s
folium-0.8.0-p 100% |################################| Time: 0:00:00  35.62 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    pandas-profiling: 1.4.1-py_1 conda-forge

pandas-profili 100% |####################

## Foursquare secret

In [3]:
# (paste secret from local file)


In [4]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version

CLIENT_ID set: True
CLIENT_SECRET set: True


## Which city?

In [6]:
CITY_INDEX=14
RADIUS=800
MAX_COORDS=1000
MAP_ZOOM_LEVEL=11
MARKER_ZOOM=0.1

## All function definitions

See notebooks 1 and 2 for details.

In [7]:
print('Starting: Get list of cities')

url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })

print('---------CITIES TABLE (raw)')
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))


l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']

print('----------CITIES DATAFRAME')
print(cities_df)


def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100, verbose=True):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request (on error try four more times before giving up)
    num_tries = 0
    results = [] # assume no venues if persistent error
    
    while num_tries < 5:
        num_tries +=1
        try:
            results_raw = requests.get(url)
            results = results_raw.json()["response"]['groups'][0]['items']
        except:
            print('(err)', end='')
            time.sleep(2) # sleep for two seconds, then retry
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(results) > 0):
        nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    if verbose:
        print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    else:
        print('{}'.format(len(results)), end='. ')
    
    return(nearby_venues)


def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[], verbose=True):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            if verbose:
                print('getting coordinate {}...'.format(this_hex))
            else:
                print('({},{}):'.format(this_hex[0], this_hex[1]), end='')
                
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius=radius, limit=limit, verbose=verbose)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start,
                       city_name='(city_name)', city_state='(city_state)',
                       radius_zoom=1.0, width=600, height=600,
                       no_touch=True,
                       zoom_control=False,
                       failsafe_abort_num=4000,
                       simple_rendering=False):
    
    if zoom_control == False:
        min_zoom = zoom_start
        max_zoom = zoom_start
    else:
        min_zoom = 0
        max_zoom = 16
    
    new_map = folium.Map(
        location=[lat_orig, long_orig],
        zoom_start=zoom_start,
        width=width,
        height=height,
        control_scale=True,
        no_touch=no_touch,
        min_zoom=min_zoom,
        max_zoom=max_zoom
    )
    
    num_markers=0

    # add markers to map
    for coords, venues in venues_dict.items():
        num_markers += 1
        if (venues.shape[0] > 0) and (num_markers < failsafe_abort_num): 
            if simple_rendering:
                folium.Circle(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    color=None,
                    fill=True,
                    fill_color='#005090',
                    fill_opacity=0.7).add_to(new_map)
            else:
                label = '{}, # venues: {}'.format(coords, venues.shape[0])
                label = folium.Popup(label, parse_html=True)
                folium.CircleMarker(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    popup=label,
                    color='blue',
                    fill=True,
                    fill_color='#3186cc',
                    fill_opacity=0.7,
                    parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; left: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map


def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100, verbose=True):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    print('[test #1 of {}] (0,0):'.format(max_coords_tested), end='')
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius, verbose=verbose)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        num_coords_tested += 1
        print('[test #{} of {}]'.format(num_coords_tested, max_coords_tested), end=' ')
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit, verbose=verbose )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state


def is_indicator_venue(venue_type):
    if venue_type is None:
        return False
    
    magic_words = [
        'yoga',
        'salad',
        'coworking',
        'alternative',
        'bike',
        'fitness',
        'running',
        'jogging',
        'cycling',
        'cycle',
        'athletics',
        'gluten',
        'health',
        'recreation',
        'tennis',
        'vegetarian',
        'vegan',
        'tennis',
        'disc golf',
        'pilates',
        'share',
        'sharing',
        'incubat',
        'innovat'
    ]
    return any(substring in venue_type.lower() for substring in magic_words)

print('Perform some tests on indicator venues:')
print(is_indicator_venue(None))
print(is_indicator_venue(''))
print(is_indicator_venue('Salad Bar'))
print(is_indicator_venue('Chinese Restaurant'))
print(is_indicator_venue('Coworking space'))


def aggregate_venues_dict(venues_dict):
    venues_agg = []
    venues_types = []
    for coord, venues in venues_dict.items():
        num_venues = venues.shape[0]
        num_indicators = 0
        if num_venues > 0:
            for this_type in venues['Venue Category'].values:
                venues_types.append(this_type)
                if is_indicator_venue(this_type):
                    num_indicators += 1
        venues_agg.append([coord, num_venues, num_indicators])
    venues_agg_df = pd.DataFrame(venues_agg)
    venues_agg_df.columns = ['coord', 'num_venues', 'num_indicators']
    return venues_agg_df, set(venues_types)


def find_venues_distance_to_center(venues_agg_df):
    '''
    Appends or updates x/y and distance-to-center columns of venues_agg_df.
    Also returns some aggregates that were calculated along the way.
    '''
    
    x, y = zip(*venues_agg_df['coord'])
    total_num_venues = venues_agg_df['num_venues'].sum()
    total_num_indicators = venues_agg_df['num_indicators'].sum()
    center_x = ( x * venues_agg_df['num_venues'] ).sum() / total_num_venues
    center_y = ( y * venues_agg_df['num_venues'] ).sum() / total_num_venues
    dx = x - center_x
    dy = y - center_y
    dist2 = dx * dx + dy * dy
    venues_agg_df['dist'] = np.sqrt(dist2)
    
    return center_x, center_y, total_num_venues, total_num_indicators


def gather_distance_distribution_stats(venues_agg_df, city_index=-1, city_display='(city name, state)', return_as_list_append_string=False):
    
    center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
    num_venues_per_coord = venues_agg_df['num_venues']
    dist_per_coord = venues_agg_df['dist']
    
    # blow up the dist_per_coord series by the nunmber of venues at the coordinate
    
    dist_list = []
    for this_coord in zip(num_venues_per_coord, dist_per_coord):
        this_list = [float(this_coord[1])] * int(this_coord[0])
        dist_list.extend(this_list)
    
    dist_list_df = pd.DataFrame(dist_list)
    dist = dist_list_df[0]
    
    # now perform statistics on that new list
    
    mean = dist.mean()
    std = dist.std()
    p25 = dist.quantile(0.25)
    median = dist.quantile(0.5)
    p75 = dist.quantile(0.75)
    max_d = dist.max()
    iqr = p75 - p25
    kurt = dist.kurtosis() # "peaky-ness"
    mad = dist.mad() # mean absolute deviation
    skew = dist.skew() # skewedness
    
    if return_as_list_append_string:
        retS = '# Create list at the beginning:\n'
        retS += '#     l = []\n\n'
        retS += 'l.append(['
        retS += '{}, "{}", ({:.2f}, {:.2f}), '.format(city_index, city_display, center_x, center_y)
        retS += '{}, {}, {}, {}, '.format(total_num_venues, total_num_indicators, mean, std)
        retS += '{}, {}, {}, {}, '.format(p25, median, p75, max_d)
        retS += '{}, {}, {}, {}'.format(iqr, kurt, mad, skew)
        retS += '])\n\n'
        retS += '# Convert list to pandas DataFrame:\n'
        retS += '#     cities_stats_df = pd.DataFrame(l)\n'
        retS += '#     cities_stats_df.columns = ['
        retS += '"city index", "city name, state", "center coord", "total num venues", "total num indicators", '
        retS += '"mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", '
        retS += '"interquartile range", "kurtosis", "mean absolute deviation", "skewedness"'
        retS += ']\n'
        return retS

    else:
        return center_x, center_y, total_num_venues, total_num_indicators, \
               mean, std, p25, median, p75, max_d,                         \
               iqr, kurt, mad, skew
    

    



Starting: Get list of cities
---------CITIES TABLE (raw)
<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></s

## Quick test

In [8]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=10, verbose=False)

print("{}, {}\n".format(city_name, city_state))

venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)

print("\n{}\n".format(venues_agg_df.head(10)))

center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew \
    = gather_distance_distribution_stats(venues_agg_df)

print(center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew)

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nlist_append_string:\n\n{}'.format(list_append_string))

make_map_from_dict(
    lat_orig, long_orig, venues_dict, 13, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)


[test #1 of 10] (0,0):51. [test #2 of 10] (-1,0):46. (1,0):64. (0,-1):47. (0,1):57. (-1,1):54. (1,-1):31. [test #3 of 10] (2,0):67. (1,1):58. (2,-1):30. [test #4 of 10] (3,0):49. (2,1):59. (3,-1):44. [test #5 of 10] (3,1):29. (2,2):21. (1,2):37. [test #6 of 10] (0,2):28. [test #7 of 10] (-1,2):33. [test #8 of 10] (-2,1):80. (-2,2):(err)26. [test #9 of 10] (-3,1):90. (-2,0):65. (-3,2):100. [test #10 of 10] (-4,2):90. (-3,3):35. (-4,3):66. St. Louis, Missouri


     coord  num_venues  num_indicators
0   (1, 2)          37               0
1   (0, 0)          51               0
2  (-1, 0)          46               0
3   (3, 0)          49               0
4  (-2, 2)          26               0
5  (-2, 1)          80               2
6   (2, 1)          59               0
7  (-3, 1)          90               3
8  (-4, 3)          66               2
9  (2, -1)          30               0

-0.636698599853 0.919675755343 1357 12 2.2747087146415454 1.0314423860681772 1.365665658909086 2.364666042

## The real deal

In [9]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=MAX_COORDS, verbose=False)

print("\n\n{}, {}\n".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
print('Total shape: {}\n\nSample 10 venues per coordinate (unsorted):'.format(venues_agg_df.shape))
print('{}\n'.format(venues_agg_df.head(10)))

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nResulting list append string for this city:\n\n{}'.format(list_append_string))


[test #1 of 1000] (0,0):51. [test #2 of 1000] (-1,0):46. (1,0):64. (0,-1):47. (0,1):57. (-1,1):54. (1,-1):31. [test #3 of 1000] (2,0):67. (1,1):58. (2,-1):30. [test #4 of 1000] (3,0):49. (2,1):59. (3,-1):44. [test #5 of 1000] (3,1):29. (2,2):21. (1,2):37. [test #6 of 1000] (0,2):28. [test #7 of 1000] (-1,2):33. [test #8 of 1000] (-2,1):80. (-2,2):26. [test #9 of 1000] (-3,1):90. (-2,0):65. (-3,2):100. [test #10 of 1000] (-4,2):90. (-3,3):35. (-4,3):66. [test #11 of 1000] (-4,1):95. (-3,0):63. [test #12 of 1000] (-5,1):17. (-4,0):28. (-5,2):48. [test #13 of 1000] (-5,3):69. [test #14 of 1000] (-6,3):9. (-5,4):34. (-6,4):20. [test #15 of 1000] (-4,4):29. [test #16 of 1000] (-2,-1):36. (-1,-1):52. [test #17 of 1000] (-3,-1):36. [test #18 of 1000] (-1,-2):(err)(err)45. (0,-2):40. [test #19 of 1000] (4,0):40. (4,-1):33. [test #20 of 1000] (-6,2):29. [test #21 of 1000] (1,-2):29. [test #22 of 1000] [test #23 of 1000] (-2,-2):30. (-1,-3):33. (0,-3):39. [test #24 of 1000] (3,-2):8. (4,-2):11. 

38. (-9,-12):13. (-10,-10):37. [test #220 of 1000] (-11,-11):18. (-10,-12):10. (-11,-10):28. [test #221 of 1000] (-10,-9):24. (-11,-9):(err)22. [test #222 of 1000] (-12,-10):27. (-12,-9):14. [test #223 of 1000] (-9,-9):22. [test #224 of 1000] (-13,-10):22. (-12,-11):34. (-13,-9):25. [test #225 of 1000] (-13,-11):28. (-12,-12):13. (-11,-12):12. [test #226 of 1000] (-14,-11):10. (-13,-12):5. (-14,-10):(err)19. [test #227 of 1000] [test #228 of 1000] (12,-7):2. (12,-8):1. [test #229 of 1000] (12,-6):4. [test #230 of 1000] [test #231 of 1000] [test #232 of 1000] (4,-13):20. (4,-14):12. [test #233 of 1000] [test #234 of 1000] (-10,0):42. (-10,1):20. [test #235 of 1000] (-11,0):48. (-10,-1):59. (-11,1):31. [test #236 of 1000] (-11,-1):40. (-10,-2):33. (-9,-2):19. [test #237 of 1000] (-12,0):37. (-12,1):29. [test #238 of 1000] (-12,-1):27. (-11,-2):21. [test #239 of 1000] (-13,0):10. (-13,1):7. [test #240 of 1000] (-10,-3):14. (-9,-3):15. [test #241 of 1000] (-11,2):22. (-12,2):12. [test #242

22. (-8,-17):21. (-7,-17):19. [test #451 of 1000] (-9,-13):8. [test #452 of 1000] (-6,-17):15. [test #453 of 1000] (-5,-16):11. (-5,-17):5. [test #454 of 1000] (-10,-16):25. (-9,-17):25. (-10,-15):24. [test #455 of 1000] (-11,-16):17. (-10,-17):21. (-11,-15):13. [test #456 of 1000] (-9,-18):10. (-8,-18):3. [test #457 of 1000] (-10,-14):8. (-11,-14):13. [test #458 of 1000] (-5,-15):9. [test #459 of 1000] (-7,-18):1. [test #460 of 1000] (-11,-17):21. (-10,-18):21. [test #461 of 1000] (-12,-17):32. (-11,-18):35. (-12,-16):11. [test #462 of 1000] (-12,-18):36. (-11,-19):24. (-10,-19):8. [test #463 of 1000] (-13,-18):45. (-12,-19):35. (-13,-17):44. [test #464 of 1000] (-14,-18):32. (-13,-19):29. (-14,-17):37. [test #465 of 1000] (-13,-16):8. (-14,-16):12. [test #466 of 1000] (-15,-17):36. (-15,-16):14. [test #467 of 1000] (-16,-17):52. (-15,-18):32. (-16,-16):41. [test #468 of 1000] (-17,-17):49. (-16,-18):59. (-17,-16):39. [test #469 of 1000] (-17,-18):48. (-16,-19):45. (-15,-19):45. [test

3. (-14,-22):5. [test #652 of 1000] (-13,-22):5. [test #653 of 1000] (-38,-7):6. (-39,-7):4. [test #654 of 1000] (-32,-12):13. [test #655 of 1000] (-34,-6):9. (-35,-6):10. [test #656 of 1000] (-50,-11):13. (-49,-12):6. [test #657 of 1000] (-51,-9):3. [test #658 of 1000] (9,-14):13. (9,-15):5. [test #659 of 1000] [test #660 of 1000] (4,2):7. [test #661 of 1000] (9,2):6. (8,2):10. [test #662 of 1000] (10,-13):10. [test #663 of 1000] (0,4):18. (-1,4):8. [test #664 of 1000] (1,4):12. (0,5):4. (-1,5):7. [test #665 of 1000] (-13,8):9. (-14,8):12. [test #666 of 1000] (-15,8):15. [test #667 of 1000] (-16,8):7. (-15,9):12. (-16,9):8. [test #668 of 1000] [test #669 of 1000] [test #670 of 1000] (-20,7):12. (-21,7):13. [test #671 of 1000] [test #672 of 1000] (-4,6):7. (-5,7):6. [test #673 of 1000] [test #674 of 1000] [test #675 of 1000] (-15,-15):9. [test #676 of 1000] (-16,7):8. [test #677 of 1000] [test #678 of 1000] [test #679 of 1000] (10,-21):1. (10,-22):3. [test #680 of 1000] (-24,-6):4. [te

32. (-40,11):38. (-41,11):36. [test #861 of 1000] (-39,11):20. [test #862 of 1000] (-38,11):6. [test #863 of 1000] (-38,7):3. (-37,7):5. [test #864 of 1000] (-40,12):7. (-41,12):4. [test #865 of 1000] (-42,11):18. (-42,12):21. [test #866 of 1000] (-33,10):8. [test #867 of 1000] (-42,10):18. (-41,9):6. [test #868 of 1000] (-40,8):4. [test #869 of 1000] (-35,6):6. [test #870 of 1000] [test #871 of 1000] (-43,12):17. (-42,13):5. (-43,13):4. [test #872 of 1000] (-39,12):5. [test #873 of 1000] (-30,11):4. [test #874 of 1000] [test #875 of 1000] (-43,11):17. [test #876 of 1000] (-43,10):22. (-42,9):2. [test #877 of 1000] (-44,10):38. (-43,9):13. (-44,11):30. [test #878 of 1000] (-45,10):43. (-44,9):28. (-45,11):43. [test #879 of 1000] (-46,10):19. (-45,9):18. (-46,11):24. [test #880 of 1000] (-45,12):17. (-46,12):4. [test #881 of 1000] (-44,12):8. [test #882 of 1000] (-44,8):4. (-43,8):6. [test #883 of 1000] (-47,11):8. (-47,12):3. [test #884 of 1000] (-47,10):4. (-46,9):4. [test #885 of 100

In [11]:
make_map_from_dict(
    lat_orig, long_orig, venues_dict, 11, city_name, city_state, radius_zoom=7,
    width=800, height=600, simple_rendering=True)
